<a href="https://colab.research.google.com/github/vrishikesh-deepedge/ee_aircraft/blob/main/ee_aircraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00


In [14]:
import ee
import os
import shutil
import cv2
from fastapi import FastAPI, Form, UploadFile, File
from fastapi.responses import JSONResponse
from starlette.middleware.cors import CORSMiddleware
import requests
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Authenticate and initialize Earth Engine
try:
    ee.Initialize(project="test-earth-engine-458605")
except ee.ee_exception.EEException:
    ee.Authenticate()
    ee.Initialize(project="test-earth-engine-458605")

app = FastAPI()

# CORS setup for frontend access
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Ensure image directory exists
os.makedirs("aircraft_images", exist_ok=True)

def fetch_satellite_image(longitude: float, latitude: float):
    point = ee.geometry.Geometry.Point([longitude, latitude])
    image_collection = (
        ee.imagecollection.ImageCollection("LANDSAT/LC08/C01/T1_SR")
        .filterBounds(point)
        .filterDate("2023-01-01", "2023-12-31")
        .sort("system:time_start")
    )
    image = image_collection.first().select(["B4", "B3", "B2"]).visualize(min=0, max=3000)
    url = image.getThumbURL({"region": point.buffer(500).bounds(), "dimensions": 256})
    return url

def download_image_from_url(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        return True
    return False

def match_images(aircraft_path, satellite_path):
    aircraft_image = cv2.imread(aircraft_path, cv2.IMREAD_GRAYSCALE)
    satellite_image = cv2.imread(satellite_path, cv2.IMREAD_GRAYSCALE)

    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(aircraft_image, None)
    kp2, des2 = orb.detectAndCompute(satellite_image, None)

    if des1 is None or des2 is None:
        return 0

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)

    matches = sorted(matches, key=lambda x: x.distance)

    img_matches = cv2.drawMatches(aircraft_image, kp1, satellite_image, kp2, matches[:10], None, flags=2)
    cv2.imshow('Matches', img_matches)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return len(matches)

@app.get("/")
async def root():
    return JSONResponse({"message": "Hello World"})

@app.post("/upload")
async def upload_image(file: UploadFile = File(...), longitude: float = Form(...), latitude: float = Form(...)):
    # Save aircraft image
    aircraft_path = f"aircraft_images/{file.filename}"
    with open(aircraft_path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    # Retrieve satellite image from GEE
    satellite_url = fetch_satellite_image(longitude, latitude)
    satellite_path = f"aircraft_images/satellite_{file.filename}"
    if not download_image_from_url(satellite_url, satellite_path):
        return JSONResponse(status_code=500, content={"error": "Failed to download satellite image"})

    # Match images
    match_score = match_images(aircraft_path, satellite_path)

    result = {
        "aircraft_image": file.filename,
        "satellite_image": satellite_path,
        "match_score": match_score,
        "status": "on track" if match_score > 100 else "off track"
    }
    return JSONResponse(content=result)

# Set up ngrok
ngrok_tunnel = ngrok.connect(8000, pyngrok_config=ngrok.PyngrokConfig(
    auth_token='1tAeUFQnC5yWXcyNyYzpdrfhQsN_4xEr8qULGSan4D4XPJfqK'
))
print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# Run the server
uvicorn.run(app, port=8000)


INFO:     Started server process [443]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://c92d-34-16-175-19.ngrok-free.app
INFO:     202.179.94.1:0 - "GET / HTTP/1.1" 200 OK
INFO:     202.179.94.1:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [443]
